### [요구사항 1]	titanic 딥러닝 모델 기본 훈련

In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from datetime import datetime
import wandb
import argparse
import os
import pandas as pd

In [2]:
from pathlib import Path
try:
    BASE_PATH = str(Path(__file__).resolve().parent.parent) # 현재 실행 중인 파일의 상위 디렉토리 두 단계(=link_dl)을 호출
except NameError: # Jupyter Notebook처럼 __file__이 없는 환경에서는 os.getcwd()로 현재 작업 디렉토리를 기준으로 설정
    BASE_PATH = str(Path(os.getcwd()).resolve().parent)
print("BASE_PATH:", BASE_PATH)

# 기본 디렉토리를 Python Path에 추가 => Python 인터프리터가 Python 라이브러리와 애플리케이션을 찾을 위치를 안내
import sys
sys.path.append(BASE_PATH)

# _03_homeworks.homework_2 디렉토리의 titanic_dataset.py의 get_preprocessed_dataset 함수를 import
from _03_homeworks.homework_2.titanic_dataset \
    import get_preprocessed_dataset

BASE_PATH: C:\Users\WJM\git\link_dl


In [3]:
def get_data():
    train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset() # titanic_dataset.py에서 전처리된 훈련, 검증, 평가용 데이터셋 호출

    # 학습, 검증, 평가용 데이터로더로 변환
    train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True) # wandb에서 설정한 배치 크기로 설정, 매 Epoch마다 데이터 순서를 섞어 학습
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset)) # 모든 검증용 데이터를 한꺼번에 번에 배치 처리
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset)) # 모든 평가용 데이터를 한꺼번에 배치 처리

    return train_data_loader, validation_data_loader, test_data_loader

### 참고: Dataset과 Dataloader의 차이
1. 데이터셋(Dataset): 테이블, 배열, CSV, JSON 등으로 구성된 원본 데이터의 모음
2. 데이터로더(Dataloader): Pytorch에서 제공하는 데이터 적재 유틸리티. 데이터셋을 사용자가 설정한 배치 크기 단위로 나누어 배치들로 만들고, training loop에서 배치 단위로 사용할 수 있는 반복 가능한 객체(iterator)로 만들어준다. Dataloader로 만들지 않으면 데이터셋의 데이터는 training loop에서 샘플 단위로 하나씩만 반복할 수 있다.

### [요구사항 2] Activation Function과 Batch Size 변경 및 선택하기

In [4]:
# 딥러닝 모델
class MyModel(nn.Module):
    def __init__(self, n_input, n_output): # titanic 데이터의 피처 개수=10, 출력 클래스(0(사망), 1(생존))의 개수=2
        super().__init__()

        activation_name = wandb.config.activation_fn
        if activation_name == "ReLU":
            activation_fn = nn.ReLU()
        elif activation_name == "Sigmoid":
            activation_fn = nn.Sigmoid()
        elif activation_name == "ELU":
            activation_fn = nn.ELU()
        elif activation_name == "LeakyReLU":
            activation_fn = nn.LeakyReLU()
        else:
            raise ValueError(f"Unsupported activation: {activation_name}")

        self.model = nn.Sequential( # 딥러닝 은닉층 구성. 층의 개수는 wandb에서 설정.
            nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]), # 첫번째 은닉층. 유닛 개수는 64개
            nn.ReLU(), # 활성화 함수(ReLU)
            nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]), # 두번째 은닉층. 유닛 개수는 32개
            nn.ReLU(),
            nn.Linear(wandb.config.n_hidden_unit_list[1], n_output), # 출력층
        )

    # 순전파: 입력값 x를 self.model로 통과시켜 출력을 생성, 즉 다음 은닉층으로 넘어가게 함.
    def forward(self, x):
        x = self.model(x)
        return x

![Activation Function과 Batch Size별 Training loss와 Validation loss 비교](https://i.imgur.com/abMQe9e.jpeg)

활성화 함수로 ReLU를 선택하고 배치 크기를 64로 설정했을 때가 Validation loss가 가장 작게 나오고, Training loss 역시 아주 높지는 않기에 최고의 조합으로 선정하였다.

In [5]:
def training_loop(model, optimizer, train_data_loader, validation_data_loader, device): # 학습할 모델, 모델의 패러미터 업데이트용 optimizer, 학습용과 검증용 Dataloader, 학습 장치를 매개변수로 받음.
    n_epochs = wandb.config.epochs # wandb에서 설정한 Epoch 수=200 => 모든 데이터를 200번 보겠음.
    loss_fn = nn.BCEWithLogitsLoss() # 이진 분류용(0 또는 1=사망 혹은 생존) 손실 함수
    best_validation_loss = float('inf') # 가장 검증 손실이 작게 나온 모델을
    best_path = os.path.join(BASE_PATH, "best_model.pth") # 최고의 모델로 보아 다음 경로에 저장함.

    for epoch in range(1, n_epochs + 1): # 1부터 200까지 한 Epoch마다 모델 훈련
        model.train() # 모델 훈련
        loss_train = 0.0
        num_trains = 0
        for train_batch in train_data_loader: # 훈련용 Dataloader에서 배치 단위=데이터 64개씩 반복
            inputs = train_batch['input'].to(device) # 입력값과
            targets = train_batch['target'].to(device).float().unsqueeze(1) # 타겟값을 device로 이동시켜서
            output_train = model(inputs) # 입력값에 따른 모델의 예측값 계산
            loss = loss_fn(output_train, targets) # 손실, 즉 타겟값과 모델의 예측값의 차이 계산

            optimizer.zero_grad() # 이전 기울기 초기화
            loss.backward() # 역전파 알고리즘으로 기울기 계산
            optimizer.step() # 패러미터 업데이트

            batch_size = inputs.size(0)
            loss_train += loss.item() * batch_size # 배치별 손실 평균값 × 배치 크기 = 배치의 총 손실
            num_trains += batch_size # 지금까지 학습한 샘플의 수
        avg_train = loss_train / num_trains if num_trains > 0 else float('nan') # 전체 학습 데이터 기준 평균 손실

        model.eval() # 모델 검증
        loss_validation = 0.0
        num_validations = 0
        with torch.no_grad(): # 기울기 계산 종료
            for validation_batch in validation_data_loader: # 검증용 데이터들로 모델 검증
                inputs = validation_batch['input'].to(device)
                targets = validation_batch['target'].to(device).float().unsqueeze(1)
                output_validation = model(inputs)
                loss = loss_fn(output_validation, targets)
                batch_size = inputs.size(0)
                loss_validation += loss.item() * batch_size
                num_validations += batch_size
        avg_validation = loss_validation / num_validations if num_validations > 0 else float('nan')

        wandb.log({ # wandb에 전체 학습/검증용 데이터별 평균 손실 기록
            "Epoch": epoch,
            "Training loss": avg_train,
            "Validation loss": avg_validation
        })

        if avg_validation < best_validation_loss: # 현재까지 최고(로 낮은) 검증 손실보다 더 낮은 검증 손실을 보이는 모델이 있다면
            best_validation_loss = avg_validation # 최고 검증 손실값을 갱신하고
            torch.save({ # 새로운 최고 성능의 모델과 그 저장 경로를 갱신하여 저장
                "Epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "Validation loss": avg_validation
            }, best_path)
            print(
                f"Epoch {epoch},"
                f"Training loss {avg_train:.4f}, "
                f"Validation loss {avg_validation:.4f}"
                f"Saved best model to {best_path}"
            )

    return best_path

In [6]:
# training loop 중에 나온 최고 성능 모델을 복원
def make_submission(checkpoint_path, model_constructor, n_input, n_output, test_data_loader, device, submission_path):
    checkpoint = torch.load(checkpoint_path, map_location=device) # 전체 학습 모델 중 가장 작은 Validation loss를 기록한 모델 호출
    model = model_constructor(n_input, n_output) # 새로운 모델 객체를 생성하여
    model.load_state_dict(checkpoint["model_state_dict"]) # 최고 성능 모델의 패러미터를 로드
    model.to(device) # device=학습 장치로 이동 후
    model.eval() # 평가

    rows = []
    with torch.no_grad(): # 평가 시 기울기 계산 비활성화
        for test_batch in test_data_loader:
            inputs = test_batch['input'].to(device) # 평가용 Dataloader의 입력값들을
            outputs = model(inputs) # 모델에 입력하여 출력값 계산
            if outputs.dim() == 2 and outputs.size(1) == 1:
                outputs = outputs.squeeze()
            probs = torch.sigmoid(outputs).cpu().numpy()
            preds = (probs >= 0.5).astype(int) # 출력값이 50% 이상이면 1(생존), 미만이면 0(사망)으로 예측

            # 평가용 데이터셋의 승객별 PassengerId를 가져와 생존/사망 여부를 기록. 만약 PassengerId가 없다면 인덱스로 Id 생성
            ids = test_batch.get('PassengerId', list(range(len(rows), len(rows)+len(preds))))
            for pid, p in zip(ids, preds):
                rows.append({"PassengerId": int(pid), "Survived": int(p)})

     # 예측 결과를 pandas DataFrame으로 변환
    df = pd.DataFrame(rows)
    df = df.astype({"PassengerId": int, "Survived": int})
    df = df[["PassengerId", "Survived"]]
    df.to_csv(submission_path, index=False) # csv로 저장
    print(f"Saved submission to {submission_path} ({len(df)} rows)") # 저장 경로와 행 개수 출력
    return submission_path

### ※ 캐글에 제출할 submission은 PassengerId와 Survived(0 혹은 1) 형태의 테이블이어야 하기 때문에 titanic_dataset.py를 일부 수정하여 평가용 데이터셋에 PassengerId 삽입

class TitanicTestDataset(Dataset):
  def __init__(self, X, passenger_ids): # ', passenger_ids' 추가
    self.X = torch.FloatTensor(X)
    self.passenger_ids = passenger_ids # 추가
    
  ...
  
  def __getitem__(self, idx):
    feature = self.X[idx]
    return {'input': feature, 'PassengerId': self.passenger_ids[idx]} # ''PassengerId': self.passenger_ids[idx]' 추가

def get_preprocessed_dataset():
    ...
    
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
    test_ids = test_df["PassengerId"].values # 원본 PassengerId를 불러오기 위해 추가한 코드
    test_dataset = TitanicTestDataset(test_X.values, passenger_ids=test_ids) # ', passenger_ids=test_ids' 추가
    #print(test_dataset)

    return train_dataset, validation_dataset, test_dataset

### [요구사항 3]	테스트 및 submission.csv 생성

In [10]:
def main(args):
    current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

    # config 딕셔너리 준비 후
    config = {
        'epochs': args.epochs,
        'batch_size': args.batch_size,
        'learning_rate': args.learning_rate,
        'n_hidden_unit_list': args.n_hidden_unit_list,
        'activation_fn': args.activation_fn
    }

    # wandb 실행
    wandb.init(
        mode="online" if args.wandb else "disabled",
        project="my_model_training",
        notes="Deep Learning and Exercise Homework2",
        tags=["my_model", "titanic"],
        name=current_time_str,
        config=config
    )
    print(args)
    print(wandb.config)

    # device 결정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 훈련, 검증, 평가용 Dataloader 생성
    train_data_loader, validation_data_loader, test_data_loader = get_data()

    print("#" * 50, 1)

    # 첫번째 배치로부터 입력값의 수를 계산
    first_batch = next(iter(train_data_loader))
    input_shape = first_batch['input'].shape
    n_input = input_shape[1]
    n_output = 1

    # MyModel 생성
    model = MyModel(n_input, n_output).to(device)
    optimizer = optim.SGD(model.parameters(), lr=config["learning_rate"])

    # MyModel의 training loop를 시행하여 최고 성능 모델 및 그 경로 반환
    best_path = training_loop(model, optimizer, train_data_loader, validation_data_loader, device)

    # 최고 성능 모델로 submission.csv 생성
    submission_path = os.path.join(BASE_PATH, "submission.csv")
    make_submission(best_path, lambda n_in, n_out: MyModel(n_in, n_out),
                    n_input, n_output, test_data_loader, device, os.path.join(BASE_PATH, "submission.csv"))
    
    wandb.finish() # wandb 종료

In [12]:
# https://docs.wandb.ai/guides/track/config
if __name__ == "__main__":
    # Argparse를 사용하여 명령어에 "--옵션"을 넣을 때만 해당 옵션을 실행하고, 아니면 각 옵션별 default를 실행
    parser = argparse.ArgumentParser()

    parser.add_argument(
        "--wandb", action=argparse.BooleanOptionalAction, default=False, help="Use wandb or not"
    )

    parser.add_argument(
        "-b", "--batch_size", type=int, default=64, help="Batch size"
    )

    parser.add_argument(
        "-e", "--epochs", type=int, default=200, help="Epochs"
    )

    parser.add_argument(
        "--learning_rate", type=float, default=1e-3, help="Learning rate"
    )

    parser.add_argument(
        "--n_hidden_unit_list", nargs="+", type=int, default=[32, 16],
        help="Hidden units list, e.g. --n_hidden_unit_list 32 16"
    )

    parser.add_argument(
        "--activation_fn", type=str, default="ReLU", choices=["Sigmoid", "ReLU", "ELU", "LeakyReLU"],
        help="Actiavation function to use in the model"
    )

    # args = parser.parse_args()
    from types import SimpleNamespace
    args = SimpleNamespace(
        wandb = True,
        batch_size = 64,
        epochs = 200,
        learning_rate = 1e-3,
        n_hidden_unit_list = [32, 16],
        activation_fn = "ReLU"
    )

    main(args)

namespace(wandb=True, batch_size=64, epochs=200, learning_rate=0.001, n_hidden_unit_list=[32, 16], activation_fn='ReLU')
{'epochs': 200, 'batch_size': 64, 'learning_rate': 0.001, 'n_hidden_unit_list': [32, 16], 'activation_fn': 'ReLU'}
Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'title', 'family_num', 'alone'],
      dtype='object')
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  title  \
0       0.0       3    1  22.0      1      0   7.2500         2      2   
1       1.0       1    0  38.0      1      0  71.2833         0      3   
2       1.0       3    0  26.0      0      0   7.9250         2      1   
3       1.0       1    0  35.0      1      0  53.1000         2      3   
4       0.0       3    1  35.0      0      0   8.0500         2      2   
5       0.0       3    1  29.0      0      0   8.4583         1      2   
6       0.0       1    1  54.0      0      0  51.8625         2      2   
7       0.0       3    1   2.

Epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇██
Training loss,█▅▄▄▄▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation loss,█▇▇▆▅▅▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
Epoch,200
Training loss,0.58211
Validation loss,0.57236


### [요구사항 4]	submission.csv제출 및 등수확인
![캐글 점수 및 순위](https://i.imgur.com/spqbUaG.png)

# 숙제 후기
wandb, Argparse 등 처음 보는 라이브러리들과 Epoch와 Batch별 훈련을 거듭할 때마다 갱신되는 Training loss와 Validation loss를 통한 최고 성능 모델 업데이트, 활성화 함수와 배치 크기별 모델 성능 비교 등 다양한 실습을 경험해볼 수 있어 매우 유익한 과제였습니다. 실제로 손실 함수값을 기준으로 여러가지 요소를 변경해가며 최고 성능을 내는 모델을 찾아가는 과정도 재미있었습니다.
한가지 아쉬웠던 점은 평가용 데이터셋까지 PassageId 피처가 제거돼 있어서 따로 titanic_dataset.py를 수정해야 했던 것이었습니다. 최대한 titanic_dataset.py는 수정하지 않고 과제를 진행하고 싶어서 샘플마다 무작위로 PassageId를 부여하는 등 해결책을 찾느라 시간을 많이 소비했는데, 캐글에 submission 제출 시 원본 PassageId가 반드시 필요하므로 처음부터 남겨져 있더라면 더 좋았을 것 같습니다.